In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
col = [ 'Class Name','Left weight','Left distance','Right weight','Right distance']
df = pd.read_csv('balance-scale.data',names=col,sep=',')
X = df.to_numpy()
X = np.delete(X, 0, 1)
y = df['Class Name'].to_numpy()
#print(X.shape)
#print(y.shape)
#print(y)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42)
print(X_train)
print(y_train)
#print(np.unique(X_train[col[1]]))
#print(X_train['Left weight'])
#asd = [x for x in X_train['Left weight'] if x > 2]
#print(y_train['Class Name'])

[[2 4 1 5]
 [1 4 3 4]
 [5 1 5 3]
 ...
 [3 1 5 1]
 [4 3 3 1]
 [1 5 1 3]]
['L' 'R' 'R' 'R' 'B' 'L' 'R' 'B' 'L' 'L' 'R' 'L' 'L' 'L' 'R' 'L' 'R' 'R'
 'B' 'R' 'R' 'R' 'L' 'R' 'R' 'R' 'R' 'L' 'L' 'L' 'R' 'L' 'L' 'R' 'R' 'R'
 'R' 'R' 'R' 'L' 'R' 'R' 'L' 'B' 'R' 'L' 'B' 'R' 'R' 'R' 'L' 'L' 'B' 'L'
 'R' 'R' 'L' 'R' 'L' 'L' 'R' 'L' 'R' 'R' 'L' 'L' 'L' 'L' 'L' 'L' 'R' 'L'
 'R' 'R' 'R' 'L' 'L' 'L' 'R' 'L' 'L' 'R' 'L' 'R' 'R' 'B' 'L' 'R' 'R' 'R'
 'R' 'R' 'L' 'R' 'L' 'L' 'L' 'L' 'L' 'L' 'R' 'L' 'R' 'L' 'L' 'L' 'L' 'B'
 'R' 'R' 'R' 'R' 'L' 'R' 'R' 'L' 'R' 'R' 'L' 'L' 'L' 'R' 'L' 'L' 'L' 'B'
 'R' 'R' 'L' 'L' 'R' 'R' 'L' 'B' 'L' 'R' 'L' 'R' 'L' 'L' 'L' 'L' 'R' 'L'
 'R' 'L' 'R' 'R' 'R' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'R' 'L' 'L' 'R' 'R'
 'R' 'L' 'L' 'L' 'R' 'L' 'L' 'R' 'L' 'R' 'R' 'R' 'L' 'L' 'R' 'R' 'L' 'R'
 'L' 'R' 'R' 'R' 'L' 'R' 'R' 'L' 'L' 'R' 'B' 'L' 'R' 'L' 'L' 'R' 'L' 'L'
 'R' 'L' 'R' 'L' 'L' 'R' 'R' 'R' 'R' 'R' 'R' 'L' 'R' 'L' 'R' 'L' 'R' 'R'
 'L' 'R' 'R' 'L' 'R' 'B' 'B' 'B' 'R' 'L' 'R' 'R' 'L'

In [4]:
from statistics import mode
from numpy import inf

class Node:#
    def __init__(self, feature=None, condition=None, left=None, right=None, value=None):#
        self.feature = feature
        self.condition = condition
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeClassifier:#
    def __init__(self, min_samples_split=1, max_depth=5, number_of_features=None):#
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.number_of_features = number_of_features

    def fit(self, X_train, y_train):#
        self.number_of_features = X_train.shape[1]
        self.root = self.build_tree(X_train, y_train)

    def predict(self, X):#
        # data reshapping
        y_predict = np.array([self.traversal(x, self.root) for x in X])
        y_predict_arr = []
        for x in y_predict:
            if (x == 0.4759725400457666):
                y_predict_arr.append('L')
            elif (x == 0.45308924485125857):
                y_predict_arr.append('R')
            elif (x == 0.07093821510297482):
                y_predict_arr.append('B')
        return np.array(y_predict_arr)

    def build_tree(self, X_train, y_train, current_depth=0):#
        number_of_samples = X_train.shape[0]
        number_of_features = X_train.shape[1]
        
        if (current_depth >= self.max_depth or self.min_samples_split > number_of_samples):
            return Node(value=mode(y_train))
        feature_split, condition_split = self.best_criteria(X_train, y_train, np.arange(0, number_of_features))
        left_branch = np.argwhere(X_train[:, feature_split] <= condition_split).flatten()
        right_branch = np.argwhere(X_train[:, feature_split] > condition_split).flatten()
        left_node = self.build_tree(X_train[left_branch, :], y_train[left_branch], current_depth + 1)
        right_node = self.build_tree(X_train[right_branch, :], y_train[right_branch], current_depth + 1)
        return Node(feature_split, condition_split, left_node, right_node)

    def best_criteria(self, X_train, y_train, features):#
        max_ig, split, condition = -inf, -inf, -inf
        for feat in features:
            # array of each possible condition for this feature
            conditions = np.unique(X_train[:, feat])
            for cond in conditions:
                gain = self.information_gain(X_train[:, feat], y_train, cond)
                # best information gain
                if gain > max_ig:
                    # records feature and condition for best IG
                    max_ig, split, condition = gain, feat, cond
        return split, condition

    def information_gain(self, X_train, y_train, cond):#
        left_branch = np.argwhere(X_train <= cond).flatten()
        right_branch = np.argwhere(X_train > cond).flatten()
        if len(left_branch) == 0 or len(right_branch) == 0:
            return 0
        pXl = len(left_branch) / len(y_train) # p(X <=)
        pXr = len(right_branch) / len(y_train) # p(X >)
        HYXl = self.entropy(y_train[left_branch]) # H(Y|X <=)
        HYXr = self.entropy(y_train[right_branch]) # H(Y|X >)
        return self.entropy(y_train) - (pXl * HYXl + pXr * HYXr) #H(Y) - H(Y|X)

    def traversal(self, root, node):#
        if node.value != None:
            return node.value
        elif root[node.feature] <= node.condition:
            return self.traversal(root, node.left)
        else:
            return self.traversal(root, node.right)
    
    def entropy(self, y):#
        probability_of_y = y
        nB, nR, nL = 0, 0, 0
        probability_of_y[probability_of_y == 'B'] = 1
        probability_of_y[probability_of_y == 'R'] = 2
        probability_of_y[probability_of_y == 'L'] = 3
        for x in probability_of_y:
            if (x == 1):
                nB += 1
            elif (x == 2):
                nR += 1
            elif (x == 3):
                nL += 1
        # p(y)
        probability_of_y[probability_of_y == 1] = nB/len(probability_of_y)
        probability_of_y[probability_of_y == 2] = nR/len(probability_of_y)
        probability_of_y[probability_of_y == 3] = nL/len(probability_of_y)
        return -np.sum([p * np.log(p) for p in probability_of_y]) # H(Y)

In [5]:
clf_model = DecisionTreeClassifier(min_samples_split=5, max_depth=3)
clf_model.fit(X_train,y_train)
y_predict = clf_model.predict(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test, y_predict)

0.6595744680851063